In [1]:
import pandas as pd
import numpy as np
from DatabaseMergingFunctions import targetLibToDF as tl
from logFunctions import write_log
from datetime import datetime

In [2]:
target_lib = "../../Decoy_Precursor_Swap/intermediate/jumplib_human_tmt18_default_target.splib"

In [3]:
targetDF = tl(target_lib)

In [4]:
targetDF_sorted = targetDF.sort_values(by=["precursorMZ"], ascending=True)

In [16]:
#mz = list(decoy_spectrumDF["m/z"])[0]
#intensity = list(decoy_spectrumDF["intensity"])[0]
#                charge = int(row[mz_cols.index("charge")]
def getDecoySpectrum_SpectraST2(sortedDF, precMZ, decoy_search, charge, scan):
    
    if scan%2 == 1:
        decoy_search == -1*decoy_search
        check_shift = precMZ+decoy_search
        check = sortedDF.query('charge == {} & precursorMZ < {}'.format(charge, check_shift)).tail(1)
        if check.shape[0] == 0:
            decoy_search == -1*decoy_search
            check_shift = precMZ+decoy_search
            check = sortedDF.query('charge == {} & precursorMZ > {}'.format(charge, check_shift)).head(1)
    else:
        check_shift = precMZ+decoy_search
        check = sortedDF.query('charge == {} & precursorMZ > {}'.format(charge, check_shift)).head(1)
        if check.shape[0] == 0:
            decoy_search == -1*decoy_search
            check_shift = precMZ+decoy_search
            check = sortedDF.query('charge == {} & precursorMZ < {}'.format(charge, check_shift)).tail(1)

    mz_cols = list(check.columns)
    arr = check.to_numpy()
    row = arr[0]
    mz = list(row[mz_cols.index("m/z")])
    intensity = list(row[mz_cols.index("intensity")])
    
    
    return mz,intensity

In [27]:
def decoySpecLibrary_Prec_Swap(df,df_sorted, specLibFolder, d, libtypename):
    mz_cols = list(df.columns)
    np_arr = df.to_numpy()

    scan_cnt = df.shape[0]
   
    proton = 1.00727646677
    write_log ("  Generating .ms2 files\n")
    now = datetime.now()
    #print (prev_mod)
    write_log("  now =", now)
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%m/%d/%Y %H:%M %p")
    year = now.strftime("%Y")
    date_time = dt_string.split() #date is 0 index, time is 1 index, AM/PM is 2 index
    
    header_ms2 = "H\tCreationDate\t"+dt_string+"\nH\tExtractor\tMakeMS2\nH\tExtractorVersion\t1.0\nH\tComments\tMakeMS2 written by Suresh Poudel, "+year+"\nH\tExtractorOptions\tMS2/MS1\n"
    new_ms2_file = specLibFolder+"/intermediate/jumplib_human_{}_decoy.splib".format(libtypename)
    # new_ms2_file = specLibFolder+"/Spectral_Library_Decoy.spLib" #modified the extension name to ms2pep
    write_log ("  Decoy spectral library is being created")
    
    cnt =1
    with open(new_ms2_file,"w") as new_ms2:
        new_ms2.write(header_ms2)
        
        for row in np_arr:
            scan = int(row[mz_cols.index("scan")])+scan_cnt
            charge = int(row[mz_cols.index("charge")])
            precMZ = float(row[mz_cols.index("precursorMZ")])
            L_ID = "Decoy_"+row[mz_cols.index("L_ID")]
            L_peptide = "Decoy_"+row[mz_cols.index("L_peptide")]
            L_protein = "Decoy_"+row[mz_cols.index("L_protein")] 
            
            precursorMZ = precMZ
            
            mz,intensity = getDecoySpectrum_SpectraST2(df_sorted, precMZ, d, charge, scan)
        

            RT = float(row[mz_cols.index("RT")])

            massNeutral = (precursorMZ*charge) - ((charge-1)*proton)
            

            new_ms2.write("S\t"+str(scan)+"\t"+str(scan)+"\t"+str(precursorMZ)+"\n")
            new_ms2.write("Z\t"+str(charge)+"\t"+str(massNeutral)+"\n")
            new_ms2.write("L\tID_with_Modification\t"+L_ID+"\t"+L_peptide+"\n")
            new_ms2.write("L\tProtein\tRepresentative\t"+L_protein+"\n")
            
            new_ms2.write("L\tMH+_Decoy\t"+str(massNeutral)+"\n")
            new_ms2.write("L\tPrecursor_Decoy\tz="+str(charge)+"\t"+str(precursorMZ)+"\n")
            new_ms2.write("L\tRT_Decoy\t"+str(RT)+"\n")

            for index, val in enumerate(mz):
                new_ms2.write(str(val)+"\t"+str(intensity[index])+"\n")
            
            if np.mod(cnt,10000) == 0:
                print ("Total scans completed {}".format(cnt))
            cnt+=1
    write_log ("  Done ...\n")

In [28]:
specLibFolder = "../../Decoy_Precursor_Swap"
libtypename = "recheck"

In [29]:
d=8
decoySpecLibrary_Prec_Swap(targetDF,targetDF_sorted, specLibFolder, d, libtypename)

  Generating .ms2 files

  now = 2021-12-01 12:27:32.508837
  Decoy spectral library is being created
Total scans completed 10000
Total scans completed 20000


KeyboardInterrupt: 